In [1]:
from konlpy.tag import Okt
try:
    import ujson
except ImportError:
    import json

In [2]:
okt = Okt()

In [3]:
with open('./data/2017_03.json') as f:
    data = json.load(f)

In [4]:
data[0]

{'date': '2017-03-01',
 'text': '플레이어 이동시키기 플레이어 닫기\n\n<앵커>\n\n\n\n98년 전 3월 1일 태극기를 들고 거리로 나서서 독립을 외치던 우리의 할아버지, 할머니들이 오늘(1일) 서울 한복판이 이렇게 둘로 갈라진 모습을 보면 뭐라고 생각하실까요? 탄핵 찬성과 반대, 각각 목소리를 높인 집회 참가자들 머리 위로는 비가 내렸고, 날씨는 을씨년스러웠습니다.\n\n\n\n먼저 권란 기자가 오늘 집회 상황 종합했습니다.\n\n\n\n<기자>\n\n\n\n광화문 광장과 맞닿은 세종대로 남쪽에서 먼저 탄핵 반대 집회가 시작됐습니다.\n\n\n\n서울광장에 이어 숭례문까지 태극기를 든 참가자들로 꽉 찼습니다.\n\n\n\n대형 태극기에다 성조기까지 등장했고, 고 박정희 전 대통령의 얼굴이 그려진 깃발도 배포됐습니다.\n\n\n\n참가자들은 헌재의 탄핵 결정을 앞두고 목청 높여 탄핵에 반대했습니다.\n\n\n\n \n\n\n\n집회 시작 네 시간쯤 뒤부터는 탄핵 반대 집회 측으로선 처음으로 청와대 방향 행진을 시작했습니다.\n\n\n\n헌재와 도심 등 5개 경로로 행진을 이어갔습니다.\n\n\n\n탄핵 반대 집회가 끝나갈 무렵인 오후 5시쯤부터는 탄핵 찬성 집회가 세종대로 북쪽 광화문광장에서 열렸습니다.\n\n\n\n충돌을 우려해 경찰이 차벽으로 양측을 막아선 가운데, 촛불을 든 시민들은 헌재의 탄핵 인용 결정을 촉구하며 목소리를 높였습니다.\n\n\n\n \n\n\n\n특히 3.1절을 맞아 위안부 피해자 이용수 할머니가 무대에 올라 아리랑을 부르자, 참가자 모두 따라 부르기도 했습니다.\n\n\n\n \n\n\n\n박근혜 대통령 탄핵 여부가 결정되는 운명의 3월 첫날에 서울 도심 광장을 둘로 나눈 대규모 집회여서 충돌이 걱정됐지만 다행히 아직 불상사는 없는 상황입니다.\n\n\n\n(영상취재 : 신진수·김성일·박진호, 영상편집 : 윤선영)',
 'title': '대통령 운명 결정할 3월…탄핵 찬반에 둘로 나뉜 3·1절'}

In [5]:
tags = set(['Noun', 'Adjective', 'Verb'])
article_list = []
for i, article in enumerate(data):
    if i % 1000 == 0:
        print(i)
    
    text = article['text'].strip()
    if len(text) > 10:
        article_list.append([w for w, t in okt.pos(text, stem=True) if t in tags and len(w) > 1])

        if len(article_list) == 10000:
            break

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [6]:
from collections import Counter

word_count = Counter()

for words_list in article_list:
    word_count.update(words_list)
    
voca = [w for w, freq in word_count.most_common(10000)]
word2id = {w: i for i, w in enumerate(voca)}

article_list_ = []
for words_list in article_list:
    words = [w for w in words_list if w in word2id]
    if len(words) >= 5:
        article_list_.append(words)

In [7]:
import numpy as np
from sklearn.preprocessing import normalize

tdm = np.zeros((len(article_list_), len(voca)), dtype=np.float32)
for i, words in enumerate(article_list_):
    for word in words:
        try:
            tdm[i, word2id[word]] += 1
        except KeyError:
            pass
        
normalize(tdm, copy=False)

array([[0.3632613 , 0.        , 0.40866897, ..., 0.        , 0.        ,
        0.        ],
       [0.55074275, 0.        , 0.27537137, ..., 0.        , 0.        ,
        0.        ],
       [0.6684272 , 0.10089468, 0.23962486, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.37038928, 0.134687  , 0.03367175, ..., 0.        , 0.        ,
        0.        ],
       [0.6489616 , 0.19468848, 0.02163205, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [8]:
from sklearn.decomposition import NMF

K = 10
nmf = NMF(n_components=K)
W = nmf.fit_transform(tdm)
H = nmf.components_

In [9]:
for k in range(K):
    print(f"{k}th topic")
    for i in H[k].argsort()[::-1][:20]:
        print(voca[i], end=' ')
    print()

0th topic
하다 국민 없다 않다 되다 우리 밝히다 아니다 돼다 대해 대통령 라며 특검 늘다 주장 대한민국 위해 한국 정치 국회 
1th topic
집회 태극기 탄핵 반대 행진 오후 참가자 경찰 단체 참석 사거리 청와대 광장 타다 대한문 촛불 들다 이다 충돌 세종 
2th topic
주다 등장 부부 스타 행복 웃음 사탕 방송사고 켈리 들다 안희정 채널 남편 지사 보다 민주 대해 출연 부인 뉴스 
3th topic
하다 대선 지원 출마 보통사람 검찰 한국 황교안 정부 인상 금리 소환 주택 청춘 수사 이다 서울 받다 선언 출시 
4th topic
탄핵 대통령 헌재 심판 선고 박근혜 되다 재판관 헌법재판소 대행 권한 결정 하다 이다 수사 인용 국회 최종 특검 변론 
5th topic
박근혜 서울 광장 광화문 열리다 퇴진 있다 기자 촛불집회 오후 행동 촉구 탄핵 대통령 국민 인용 참가자 촛불 구속 시민 
6th topic
있다 되다 않다 이다 보다 중국 기자 나오다 없다 상황 대한 때문 대선 경우 사드 아니다 돼다 결정 보수 한국 
7th topic
탄핵 서울 기각 있다 대통령 국민 궐기 운동 본부 뉴스 열리다 반대 주최 기자 집회 타다 오후 사진 취재 공동 
8th topic
대표 민주당 대선 의원 지사 문재인 더불다 탄핵 주자 후보 정당 안희정 하다 참석 한국 국민의당 자유 이재명 지지율 토론회 
9th topic
그래프 뉴스 뜨다 실시간 이다 기사 보기 많다 사람 지금 얼마나 시각 인기 나타내다 접근 하다 기자 입력 전북 헌재 
